In [1]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [2]:
df = pd.read_excel("testing.xlsx")
'''
class_names = ['negative', 'neutral', 'positive']
ax = sns.countplot(df.sentiment)
plt.xlabel('review sentiment')
ax.set_xticklabels(class_names);
'''

"\nclass_names = ['negative', 'neutral', 'positive']\nax = sns.countplot(df.sentiment)\nplt.xlabel('review sentiment')\nax.set_xticklabels(class_names);\n"

In [4]:
PRE_TRAINED_MODEL_NAME = 'cahya/bert-base-indonesian-522M'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
sample_txt = 'aku sedang makan di rumah'
tokens = tokenizer.tokenize(sample_txt)
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(f' Sentence: {sample_txt}')
print(f'   Tokens: {tokens}')
print(f'Token IDs: {token_ids}')

 Sentence: aku sedang makan di rumah
   Tokens: ['aku', 'sedang', 'makan', 'di', 'rumah']
Token IDs: [4384, 2871, 2464, 1495, 2177]


In [5]:
tokenizer.sep_token, tokenizer.sep_token_id


('[SEP]', 1)

In [6]:
tokenizer.cls_token, tokenizer.cls_token_id


('[CLS]', 3)

In [7]:
tokenizer.pad_token, tokenizer.pad_token_id


('[PAD]', 2)

In [8]:
tokenizer.unk_token, tokenizer.unk_token_id


('[UNK]', 0)

In [10]:
encoding = tokenizer.encode_plus(
  sample_txt,
  max_length=512,
  add_special_tokens=True, # Add '[CLS]' and '[SEP]'
  return_token_type_ids=False,
  pad_to_max_length=True,
  return_attention_mask=True,
  return_tensors='pt',  # Return PyTorch tensors
)

In [11]:
print(encoding.keys())

dict_keys(['input_ids', 'attention_mask'])


In [12]:
print(len(encoding['input_ids'][0]))
encoding['input_ids'][0]

512


tensor([   3, 4384, 2871, 2464, 1495, 2177,    1,    2,    2,    2,    2,    2,
           2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
           2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
           2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
           2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
           2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
           2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
           2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
           2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
           2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
           2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
           2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
           2,    2,    2,    2,    2,   

In [13]:
token_lens = []
for txt in df.berita:
  tokens = tokenizer.encode(txt, max_length=512)
  token_lens.append(len(tokens))
'''
sns.distplot(token_lens)
plt.xlim([0, 1000]);
plt.xlabel('Token count');
'''

"\nsns.distplot(token_lens)\nplt.xlim([0, 1000]);\nplt.xlabel('Token count');\n"

In [14]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
MAX_LEN = 512
class GPReviewDataset(Dataset):
  def __init__(self, reviews, targets, tokenizer, max_len):
    self.reviews = reviews
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
  def __len__(self):
    return len(self.reviews)
  def __getitem__(self, item):
    review = str(self.reviews[item])
    target = self.targets[item]
    encoding = self.tokenizer.encode_plus(
      review,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
    )
    return {
      'review_text': review,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

In [15]:
df_train, df_test = train_test_split(
  df,
  test_size=0.1,
  random_state=RANDOM_SEED
)
df_val, df_test = train_test_split(
  df_test,
  test_size=0.5,
  random_state=RANDOM_SEED
)


In [16]:
df_train.shape, df_val.shape, df_test.shape


((72, 2), (4, 2), (5, 2))

In [17]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = GPReviewDataset(
    reviews=df.berita.to_numpy(),
    targets=df.sentiment.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )
  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4
  )
BATCH_SIZE = 16
train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

In [19]:
val_data_loader


In [ ]:
print("memulai")
data = next(iter(train_data_loader))
data.keys()
print("selesai")

memulai
